# 10x Environment And Dependency Management With Python Poetry

## H2: Introduction
Brief overview of dependency management in Python and the challenges with traditional tools (pip, virtualenv).

Introduction to Poetry as a modern tool for managing dependencies and Python project environments.

Key benefits of using Poetry: simplicity, dependency resolution, and ease of configuration.


## Setting Up Python Poetry

You will mostly interact with Poetry as a command-line (CLI) tool so it makes sense to install it system-wide on your machine. This section covers this crucial first step and how to set some default configurations for Poetry based on your needs.

### Installing Poetry

You can install poetry using an official installer script which can be downloaded and run with a single command.

For macOS, Linux and WSL2:

```bash
$ curl -sSL https://install.python-poetry.org | sudo python3 -
```

For Windows Powershell (run with admin privileges):

```bash
$ (Invoke-WebRequest -Uri https://install.python-poetry.org -UseBasicParsing).Content | py -
```

If you are Windows and have installed Python from the Microsoft Store (for some weird reason), then replace `py` with `python` in the command above. 

After the installation script finishes, it prints a message asking you to add Poetry to your PATH so that the `poetry` command is available from everywhere.

For macOS, Linux and WSL2, add the following line to your shell script like `.bashrc` or `.zshrc`:

```bash
$ export PATH="/Users/bexgboost/.local/bin:$PATH"
```

For Windows, you can follow the output instructions.

Afterwards, verify your installation by running `poetry --version`. 

### Configuring Poetry

Most of Poetry's configuration is about how virtual environments are created and how packages are installed. You can print a (nearly) full list of Poetry configurations with:

```bash
$ poetry config --list
```

The output will look something like below:

```yaml
cache-dir = "/Users/bexgboost/Library/Caches/pypoetry"
experimental.system-git-client = false
installer.max-workers = null
installer.modern-installation = true
installer.no-binary = null
installer.parallel = true
keyring.enabled = true
solver.lazy-wheel = true
virtualenvs.create = true
virtualenvs.in-project = null
virtualenvs.options.always-copy = false
virtualenvs.options.no-pip = false
virtualenvs.options.no-setuptools = false
virtualenvs.options.system-site-packages = false
virtualenvs.path = "{cache-dir}/virtualenvs"  # /Users/bexgboost/Library/Caches/pypoetry/virtualenvs
virtualenvs.prefer-active-python = false
virtualenvs.prompt = "{project_name}-py{python_version}"
warnings.export = true
```

In the first line, we see a path to the Poetry cache. It is mainly used to store downloaded package distributions and virtual environments. Any virtual environments you create are stored here, by default. If you wish to change that, you can run the following command:

```bash
$ poetry config virtualenvs.path path/to/new/virtualenv/dir
```

Another important configuration is the number of cores used during installations. By default, it is set to four but we can make it faster by utilizing all CPU cores. First, find out your machine's number of cores by running `os.cpu_count()` in the Python Interpreter. Then, set the output:

```bash
$ poetry config installer.max-workers = 10 
```

One optional configuration is whether to create virtual environments in your working directory or in the cache. This is controlled via `virtualenvs.in-project` option. If you set it to True, `.venv` directory will always be created in your working directory:

```bash
$ poetry config virtualenvs.in-project true
```

## Creating a New Project with Poetry

Let's dive into the key steps of creating a new Poetry project and understanding its core configuration file.

### Initializing a new Poetry project
 


Poetry uage usually starts with creating a new project with the `poetry new` command:

```bash
$ poetry new explore-poetry
$ cd explore-poetry 
```

The command creates an `explore-poetry` directory pre-populated with the following files and directories:

```bash
explore-poetry
├── pyproject.toml
├── README.md
├── explore-poetry
│   └── __init__.py
└── tests
    └── __init__.py
```

This directory structure follows Python best practices:
- `pyproject.toml`: The main configuration file that defines project metadata and dependencies
- README.md: Documentation file explaining the project
- `explore-poetry/`: Source code directory containing the main package code, with `__init__.py` to make it a package
- `tests/`: Directory for test files, with `__init__.py` to make it a package (importable)

### Understanding `pyproject.toml`


Here, the `pyproject.toml` file demands a careful attention as it is the only non-empty file generated by Poetry:


```bash
[tool.poetry]
name = "explore-poetry"
version = "0.1.0"
description = ""
authors = ["BexTuychiev <bex@ibexprogramming.com>"]
readme = "README.md"

[tool.poetry.dependencies]
python = "^3.8"


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"
```

This file orchestrates your project and its dependencies. It uses [Tom's Obvious, Minimal Language](https://en.wikipedia.org/wiki/TOML), which was agreed as the standard configuration language for Python packages in [PEP 518](https://peps.python.org/pep-0518/). 

All `pyproject.toml` files are divided up into sections called _tables_ using brackets like `tool.poetry` or `build-system`. Poetry uses these tables to manage dependencies, project build requirements or performing other tasks. 

As you run Poetry commands outlined in the coming sections, the `pyproject.toml` file will be automatically updated.

## Working with Virtual Environments in Poetry

In this section, you will find out the ins and outs of managing virtual environments in Poetry once you set it up. You will learn why virtual environments matter, how Poetry handles them automatically and basic commands to manipulate them like creation, activation and switching. You will also learn how to manage Python versions with Poetry.

### Creating and activating a virtual environment

When you first install Poetry, it comes with no built-in environments, as evidenced by a blank output when you run `poetry env list`:

```bash
$ poetry env list  # no output
```

But as soon as you start adding dependencies (more on that later) with `poetry add package-name`, it automatically creates an environment in the cache directory for the project. For example, let's try adding `requests` as a dependency:

```bash
$ poetry add requests
```

You should receive an output similar to the following:

```python
Creating virtualenv explore-poetry--I9GJYUn-py3.8 in /Users/bexgboost/Library/Caches/pypoetry/virtualenvs
Using version ^2.32.3 for requests

Updating dependencies
Resolving dependencies... (2.5s)

Package operations: 5 installs, 0 updates, 0 removals

  - Installing certifi (2024.8.30)
  - Installing charset-normalizer (3.4.0)
  - Installing idna (3.10)
  - Installing urllib3 (2.2.3)
  - Installing requests (2.32.3)

Writing lock file
```

The first line indicates that Poetry created the environment in the cache. The next lines inform that the dependencies for `requests` are successfully resolved and a lock file is generated (more on that later too). 

Pay attention to the environment name: `explore-poetry--I9GJYUn-py3.8`. The name includes the project directory name, followed by a unique ID and then the Python version the environment is using. 

Next time you add a dependency, Poetry automatically uses this environment to install packages:

```bash
$ poetry add beautifulsoup4
Using version ^4.12.3 for beautifulsoup4

Updating dependencies
Resolving dependencies... (1.1s)

Package operations: 2 installs, 0 updates, 0 removals

  - Installing soupsieve (2.6)
  - Installing beautifulsoup4 (4.12.3)

Writing lock file
```

When you are using Poetry, your terminal won't show which Poetry virtual environment is active. To see this information, you need to run `poetry env list`:

```bash
$ poetry env list
explore-poetry--I9GJYUn-py3.8 (Activated)
```

To interact with the active Poetry environment, you can run `poetry shell`:

```bash
$ poetry shell
```

The command opens a new shell session within the current one and you can execute commands like `python` or `pytest`. You can exit the shell by typing `exit`. 

### Configuring the Python version

When you run `poetry add` or `poetry install`, Poetry automatically uses the Python version specified in your `pyproject.toml` file. To specify a different Python version, you can run `env use` command:

```bash
$ poetry env use python3.11
```

> Note that Python 3.11 must be installed system-wide on your machine for the command to work.

Check that Poetry is using the environment with the new Python version:

```bash
$ poetry env list
explore-poetry--I9GJYUn-py3.11 (Activated)
explore-poetry--I9GJYUn-py3.8
```

Notice how the new environment is automatically attached to our project (the IDs are the same). Once you configure your Python version, you can remove environments with other versions to free up disk space:

```bash
$ poetry env remove python3.8
```

When deleting an environment, you only have to specify the Python version. You can also delete all environments and start from scratch:

```bash
$ poetry env remove --all
```

Do note that `--all` tag removes all environments associated with the current project, not environments of other projects.

If you are working on team projects, it is often better to keep the virtual environment in the project directory:

```bash
$ poetry config virtualenvs.in-project true
```

> This command won't have any effect if you already have an environment for the project in the cache. To create a local environment, remove all existing ones from the cache first.

This creates a `.venv` folder in your working directory, which you should add to your `.gitignore` file.

## Managing Dependencies with Poetry

Dependency management is where Poetry shines. It provides comprehensive features to specify, install and manage dependencies so that you never run into hellish dependency conflicts. 

We will explore these features one-by-one in this section.

H3: Adding dependencies

Using poetry add to install specific packages, including specifying version constraints.

Overview of optional dependencies and their use cases.

### The syntax for specifying dependencies

H3: Handling dev dependencies

Explanation of dev dependencies and how to add them with poetry add --dev.

Typical dev dependencies for Python projects (e.g., linters, test libraries).



H3: Removing dependencies

Using poetry remove to cleanly uninstall packages.

Best practices for updating pyproject.toml after removing dependencies.


## H2: Running and Managing Scripts

H3: Defining custom scripts in pyproject.toml

Adding scripts to pyproject.toml for easy command execution.

Examples: run commands for tests, formatting, or other common tasks.

H3: Running scripts with Poetry

Executing custom scripts and commands through Poetry.

Tips for managing and organizing project-specific commands.


## H2: Comparing Python Poetry vs Pip

H3: Key differences between Poetry and pip

Explanation of pip's focus vs. Poetry’s holistic approach to project and dependency management.

Comparison of dependency resolution, virtual environment management, and ease of configuration.

H3: When to use Poetry over pip

Practical guidance on choosing between pip and Poetry for different project types and scenarios.


## H2: Publishing a Project with Poetry

H3: Preparing a project for publication

Setting versioning and metadata in pyproject.toml for a polished release.

H3: Publishing to PyPI

Steps to package a project and publish it to PyPI with poetry publish.

Additional options for private repositories.


## H2: Troubleshooting Common Issues in Poetry

Common installation or configuration issues and how to resolve them.

Dependency conflicts and strategies for resolution using Poetry’s built-in tools.


## H2: Conclusion and Next Steps
